In [2]:
import os
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import csv 
import datetime


pd.set_option('display.max_rows', 10)

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import * 
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

- Baseline file used: CJARS_data_docs_2021_p163_p169.pdf
- This file contains the broad classifications of all the charges.

- Methodology
    1. Check for occurences of crimes using broad terms mentioned in CJARS document. 
        - Example: use str.contains to check occurences of keyword 'murder'.
        - If all the occurences on checking categorize correctly into 'violent charges' in this case, mark all those records with 'violent crimes' categorization.
        - One way to do this is: wa_df['Type_Of_Crime'] = np.where(wa_df['Law_Description'].str.contains('murder'), 'violent crimes', wa_df.Type_Of_Crime) #hit and run unattended vehicle
    2. After checking for keywords from the CJARS document, split the charges into keywords and observe the occurences.
        - Example: 'child abuse' can be searched independently as 'child' and 'abuse'.
        - Now go through these occurences and find patterns and conditions to mark them into their respective crime classifications.
            - For example, on searching strings with the word 'child', crimes such as sexual assualt on child shows up. We can mark all charges containing keywords 'child' and 'sex' as 'violent'.
        - Repeat this step for all different categorizations.
    3. Check for charges by shortening the charge names and looking out for spelling mistakes due to poor record keeping.
        - Example: Assault also occurs as 'aslt', 'assualt', 'asslt', etc.

- 6 main Crime Classifications:
    1. Violent crimes
    2. Public Order crimes
    3. Drug crimes
    4. Traffic crimes
    5. Dui crimes
    6. Property crimes

In [3]:
%%time
ca_df = pd.read_csv('CA Charges by Frequency - Sheet1.csv', error_bad_lines = False)
ca_df.head()

CPU times: user 9.94 ms, sys: 5.13 ms, total: 15.1 ms
Wall time: 16.9 ms


,Charge,Count,Percent
0,NO ARREST RECEIVED,352530,6.39%
1,23152(A) VC-DUI ALCOHOL/DRUGS,186627,3.38%
2,23152(B) VC-DUI ALCOHOL/0.08 PERCENT,169502,3.07%
3,SEE COMMENT FOR CHARGE,139560,2.53%
4,11377(A) HS-POSSESS CONTROLLED SUBSTANCE,129035,2.34%


In [4]:
ca_df.shape[0]

4864

In [5]:
ca_df.Charge.nunique()

4864

In [4]:
ca_df['Charge'] = ca_df['Charge'].str.lower()

In [5]:
ca_df.head()

,Charge,Count,Percent
0,no arrest received,352530,6.39%
1,23152(a) vc-dui alcohol/drugs,186627,3.38%
2,23152(b) vc-dui alcohol/0.08 percent,169502,3.07%
3,see comment for charge,139560,2.53%
4,11377(a) hs-possess controlled substance,129035,2.34%


In [6]:
ca_df["Crime_Classification"] = ''

In [7]:
ca_df.head()

,Charge,Count,Percent,Crime_Classification
0,no arrest received,352530,6.39%,
1,23152(a) vc-dui alcohol/drugs,186627,3.38%,
2,23152(b) vc-dui alcohol/0.08 percent,169502,3.07%,
3,see comment for charge,139560,2.53%,
4,11377(a) hs-possess controlled substance,129035,2.34%,


#### Checking for violent crimes

In [9]:
print(list(ca_df[ca_df['Charge'].str.contains('pc-murder', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('manslaughter', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('kidnapping', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('robbery', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('carjacking', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('assault', na=False)].Charge.unique()))
print()
print(list(ca_df[ca_df['Charge'].str.contains('battery', na=False)].Charge.unique()))
print()

['187(a) pc-murder', '187 pc-murder', '187(a) pc-murder:first degree', '187(a) pc-murder:second degree', '187 pc-murder:second degree', '187 pc-murder:first degree', '187(a) pc-murder:first deg:shoot from vehicle', '187(a) pc-murder:sec deg on peace officer', '189 pc-murder', '189 pc-murder:first degree', '189 pc-murder:second degree', '190 pc-murder', '187/190.2 pc-murder first/spec circumstance']

['192(a) pc-voluntary manslaughter', '192(b) pc-involuntary manslaughter', '192.1 pc-voluntary manslaughter', '192 pc-manslaughter', '192.3 pc-manslaughter/vehicle', '192.2 pc-involuntary manslaughter', '192(c) pc-manslaughter/vehicle', '193(a) pc-voluntary manslaughter', '193(c) pc-vehicle manslaughter', '193 pc-manslaughter', '193(b) pc-involuntary manslaughter', 'homicide/manslaughter', '182 pc-conspiracy/homicide/manslaughter']

['207(a) pc-kidnapping', '207 pc-kidnapping', '209(b) pc-kidnapping to commit robbery', '209(a) pc-kidnapping for ransom', '209 pc-kidnapping for ransom', '207(

In [180]:
# Classifying violent crimes

In [8]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('pc-murder'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('mansl'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('kidnap'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('robbery'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('carjacking'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('assault'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('battery'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('rape'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('sex'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('abuse'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('threat'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('inj'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('homicide'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('violent'),
                                                      'Violent', ca_df.Crime_Classification)

ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('trafficking'),
                                                      'Violent', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('extortion'),
                                                      'Violent', ca_df.Crime_Classification)

ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('armed'),
                                                      'Violent', ca_df.Crime_Classification)

In [9]:
ca_df.loc[ca_df['Crime_Classification'] == 'Violent']

,Charge,Count,Percent,Crime_Classification
12,242 pc-battery,66240,1.20%,Violent
13,273.5(a) pc-inflict corporal inj spouse/cohab,64477,1.17%,Violent
24,211 pc-robbery,43833,0.79%,Violent
28,273.5 pc-inflict corporal inj on spouse/cohab,38858,0.70%,Violent
43,422 pc-threaten crime with intent to terrorize,27907,0.51%,Violent
...,...,...,...,...
4833,290.006 pc-court order reg/finding:sex grat,1,0.00%,Violent
4835,290.006 pc-290 viol:sexual grat/compulsion,1,0.00%,Violent
4837,290(g)(5) pc-sex viol predator fail verify reg,1,0.00%,Violent
4838,290(f)(3) pc-sex reg fail rpt name chng to lea,1,0.00%,Violent


In [7]:
#ca_df.to_csv('file_name.csv', sep='\t', index=False)

In [205]:
# Classifying property crimes

In [10]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('arson'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('property'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('burglary'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('forgery'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('theft'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('vandalism'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('fraud'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('financial'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('trespass'),
                                                      'property', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('destruct'),
                                                      'property', ca_df.Crime_Classification)

In [16]:
ca_df.loc[ca_df['Crime_Classification'] == 'property']

,Charge,Count,Percent,Crime_Classification
5,459 pc-burglary,124904,2.27%,property
15,496(a) pc-receive/etc known stolen property,58093,1.05%,property
19,10851(a) vc-take veh w/o own consent/veh theft,53832,0.98%,property
25,488 pc-petty theft,43589,0.79%,property
31,484(a) pc-theft,37780,0.69%,property
...,...,...,...,...
4676,498(b)(1) pc-theft divert utility srvs $950+,1,0.00%,property
4715,286(f)(3) pc-sodomy:vict unaware:perpet fraud,1,0.00%,property
4764,261(a)(4)(c) pc-rape:victim of perp fraud,1,0.00%,property
4785,31411 cc-fraud of franchise,1,0.00%,property


In [207]:
# Classifying drug crimes

In [17]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('heroin'),
                                                      'drug', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('cocaine'),
                                                      'drug', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('prescription'),
                                                      'drug', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('controlled'),
                                                      'drug', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('marijuana'),
                                                      'drug', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('paraphernalia'),
                                                      'drug', ca_df.Crime_Classification)

In [18]:
ca_df.loc[ca_df['Crime_Classification'] == 'drug']

,Charge,Count,Percent,Crime_Classification
4,11377(a) hs-possess controlled substance,129035,2.34%,drug
39,11359 hs-possess marijuana for sale,28867,0.52%,drug
61,11364(a) hs-possess unlaw paraphernalia,18713,0.34%,drug
65,11351.5 hs-poss/purchase cocaine base f/sale,16556,0.30%,drug
68,11377 hs-possess controlled substance,16414,0.30%,drug
...,...,...,...,...
4526,11162 hs-prescriptions must conform to law,1,0.00%,drug
4530,11161 hs-write unauthorized prescription,1,0.00%,drug
4566,4237 bp-forged prescription,1,0.00%,drug
4604,11166.05 hs-possess prescription blanks,1,0.00%,drug


In [209]:
# Classifying DUI crimes

In [19]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('driving'),
                                                      'DUI', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('dui'),
                                                      'DUI', ca_df.Crime_Classification)

In [20]:
ca_df.loc[ca_df['Crime_Classification'] == 'DUI']

,Charge,Count,Percent,Crime_Classification
1,23152(a) vc-dui alcohol/drugs,186627,3.38%,DUI
2,23152(b) vc-dui alcohol/0.08 percent,169502,3.07%,DUI
22,14601.2(a) vc-drive:lic susp/etc:dui:spec viol,48960,0.89%,DUI
32,23152(a)/23152(b) vc-dui:alcohol/drugs,36643,0.66%,DUI
42,23152(a) vc-dui alcohol,28109,0.51%,DUI
...,...,...,...,...
4207,502 vc-misdemeanor drunk/narcotic driving,2,0.00%,DUI
4320,23106(b) vc-dui any drug not on highway,1,0.00%,DUI
4549,38317 vc-reckless driving off-hwy veh:w/injury,1,0.00%,DUI
4612,506(a) vc-dui narcotic drugs,1,0.00%,DUI


In [40]:
# Public order crimes

In [21]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('riot'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('custody'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('prosecution'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('weapons'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('habitual offender'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('parole'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('probation'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('contempt'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('court order'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('morals'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('decency'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('immigration'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('obstruction'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('resist'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('privacy'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('invasion'),
                                                      'public order crime', ca_df.Crime_Classification)

ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('commercialize'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('deliquency'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('disorder'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('liqour'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('law violation'),
                                                      'public order crime', ca_df.Crime_Classification)

ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('tax'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('bribe'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('conflict of interest'),
                                                      'public order crime', ca_df.Crime_Classification)
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('public order offense'),
                                                      'public order crime', ca_df.Crime_Classification)

In [22]:
ca_df.loc[ca_df['Crime_Classification'] == 'public order crime']

,Charge,Count,Percent,Crime_Classification
11,violation of parole,68178,1.24%,public order crime
18,3056 pc-violation of parole:felony,55014,1.00%,public order crime
26,647(f) pc-disorderly conduct:intox drug/alcoh,40545,0.74%,public order crime
30,1203.2 pc-probation viol:rearrest/revoke,37897,0.69%,public order crime
37,647(b) pc-disorderly conduct:prostitution,29476,0.53%,public order crime
...,...,...,...,...
4643,601(a) wi-mnr violate juvenile court order,1,0.00%,public order crime
4682,279 pc-conceal child/violate custody order,1,0.00%,public order crime
4766,26 7201 us-evade/etc federal taxes,1,0.00%,public order crime
4798,639 pc-bribe employee to obtain loan,1,0.00%,public order crime


In [42]:
# Criminal Traffic

In [23]:
print(list(ca_df[ca_df['Charge'].str.contains('traffic', na=False)].Charge.unique()))

['traffic violation', 'traffic offense', '14601.3(a) vc-habitual traffic offender', '21657 vc-disobey designated traffic direction', 'traffic inf vc-traffic violations', '21804(a) vc-fail yield to traffic:hwy entrance', '21655.5 vc-disobey traffic lane signs', '21461.5 vc-pedestrian fail obey traffic signs', '236.1(a) pc-human trafficking/force labor', '236.1(c) pc-human trafficking/-18yrs', '236.1(b) pc-human traffic:deprv prsnl liberty', '42005(e) vc-fail to attend:traffic school', '36 4.12 us-fed reg:traffic cntl device viol', '22517 vc-vehicle door open to traffic', '21367(b) vc-disobey construction traffic cntl', '21462 vc-all traffic/etc obey signals', '21100.3 vc-fail to obey traffic regulator', '42005(g) vc-fail to attend traffic school', '21464(c) vc-intrfer w/traffic device:death/inj', '21465 vc-unauth dsply traffic device/sign', '18 2318(a) us-trafficking counterfeit labels']


In [24]:
ca_df['Crime_Classification'] = np.where(ca_df.Charge.str.contains('traffic'),
                                                      'Criminal Traffic', ca_df.Crime_Classification)

In [25]:
ca_df.loc[ca_df['Crime_Classification'] == 'Criminal Traffic']

,Charge,Count,Percent,Crime_Classification
141,traffic violation,6725,0.12%,Criminal Traffic
469,traffic offense,974,0.02%,Criminal Traffic
555,14601.3(a) vc-habitual traffic offender,701,0.01%,Criminal Traffic
950,21657 vc-disobey designated traffic direction,254,0.00%,Criminal Traffic
961,traffic inf vc-traffic violations,246,0.00%,Criminal Traffic
...,...,...,...,...
3150,21100.3 vc-fail to obey traffic regulator,7,0.00%,Criminal Traffic
3618,42005(g) vc-fail to attend traffic school,4,0.00%,Criminal Traffic
3621,21464(c) vc-intrfer w/traffic device:death/inj,4,0.00%,Criminal Traffic
4116,21465 vc-unauth dsply traffic device/sign,2,0.00%,Criminal Traffic


In [44]:
# Not known/missing

In [27]:
print(list(ca_df[ca_df['Charge'].str.contains('felony', na=False)].Charge.unique()))
print()


print(list(ca_df[ca_df['Charge'].str.contains('misdemeanor', na=False)].Charge.unique()))
print()




['3056 pc-violation of parole:felony', '1320(b) pc-failure to appear on felony charge', '667.5(b) pc-pr prison:new felony:enhance term', '978.5 pc-bench warrant:fta:felony charge', '1320.5 pc-failure to appear on felony charge', '979 pc-bench warrant:fail appear:felony chg', '367e pc-felony drunk driving', '667(a)(1) pc-prior felony conviction', '290(g)(2) pc-fail reg/etc felony sex off/pr', '667(a) pc-prior felony conviction', '12025(b)(1) pc-ccw in vehicle w/pr felony conv', '667 pc-prior felony conviction', '647f pc-felony prostitution', '25400(c)(1) pc-ccw in vehicle w/pr felony conv', '12023(a) pc-carry loaded firearm:commit felony', '25800(a) pc-carry loaded firearm:commit felony', '12021.1(c) pc-pos/etc f/arm:specific felony', '182.5 pc-conspiracy commit felony:gang member', '4532(b)(1) pc-escape:felony charge pending', '23175.5 vc-dui within 10 yrs of pr felony dui', '12023 pc-commit felony w/dangerous weapon', '221 pc-assault to commit other felony', '222 pc-give drugs to aid 

In [ ]:
# Save to csv 

In [28]:
ca_df.to_csv('CA_Classify.csv', sep='\t', index=False)